# Class Decorators

Class decorators are classes that include functions as state.

Defining a simple class decorator.

In [1]:
class MyClassDecorator:

    def __init__(self, func):
        print("in __init__")
        self.func = func
        self.some_state = 0

    def __call__(self, *args, **kwargs):
        print("in __call__")
        print("do your decorating of args and kwargs here!")
        # update some state
        self.some_state += 1
        self.func(*args, **kwargs)

Let's define a non-decorated function...

In [2]:
def my_function():
    print("in my_function")

Decorate the function using the class constructor...

In [3]:
decorated_function = MyClassDecorator(my_function)
print("type(decorated_function)=", type(decorated_function))        # decorated function is in fact a class instance
print("now call the decorated function")
decorated_function()                                                # call the instance -> __call__ for "()" operator

in __init__
type(decorated_function)= <class '__main__.MyClassDecorator'>
now call the decorated function
in __call__
do your decorating of args and kwargs here!
in my_function


Decorating my_function with the class decorator using @...

In [4]:
@MyClassDecorator                                                   # with parameters this is effectively...
def my_function():                                                  # ...MyClassDecorator(my_function)
    print("in my_function")

print("call my_function")
my_function()

in __init__
call my_function
in __call__
do your decorating of args and kwargs here!
in my_function


Decorating my_function without parameters...

In [5]:
class MyParameterisedClassDecorator:

    class MyLocalClassDecorator:                                      # you can create local class within class
        def __init__(self, func, a_parameter=False):                  # could have made it an external base class
            print("in MyLocalClassDecorator.__init__")                # so get_state() could be overloaded
            self.func = func
            self.parameter = a_parameter
            self.some_state = 0

        def __call__(self, *args, **kwargs):
            print("in MyLocalClassDecorator.__call__")
            print("do your decorating of args and kwargs here!")
            # update some state
            if self.parameter:
                self.some_state += 1
            return self.func(*args, **kwargs)

        def get_state(self):
            return self.parameter, self.some_state

    def __init__(self, _func=None, *, a_parameter=False):
        print("in MyParameterisedClassDecorator.__init__")
        self.parameter = a_parameter
        self.func = None
        if _func is not None:
            # decorator has been created like this MyParameterisedClassDecorator(my_function)
            print("no parameters have been specified as _func is", _func)
            self.func = MyParameterisedClassDecorator.MyLocalClassDecorator(_func, a_parameter=self.parameter)

    def __call__(self, *args, **kwargs):
        print("in MyParameterisedClassDecorator.__call__")
        if self.func is None:     # has been called like this MyParameterisedClassDecorator(a_parameter=2)(my_function)
            if len(args) != 1 or not callable(args[0]):
                raise ValueError("expecting only a single function to decorate!")
            self.func = MyParameterisedClassDecorator.MyLocalClassDecorator(args[0], a_parameter=self.parameter)
            return self.func
        return self.func(*args, **kwargs)

    def get_state(self):
        # in case where parameters are used my_function will be a MyParameterisedClassDecorator object
        if self.func is None:
            raise ValueError("undecorated function!")
        return self.func.get_state()

@MyParameterisedClassDecorator
def my_function(my_function_parameter):
    print("in my_function:", my_function_parameter)

print("calling my_function")
my_function('non-parameterised class')
print("my_function is an MyParameterisedClassDecorator so can access as such")
print("type(my_function) =", type(my_function))
print("my_function.func.parameter =", my_function.func.parameter)
print("my_function.func.state =", my_function.func.some_state)
print("my_function.get_state() =", my_function.get_state())         # use a method to get state

in MyParameterisedClassDecorator.__init__
no parameters have been specified as _func is <function my_function at 0x000001EFEDDEFAC0>
in MyLocalClassDecorator.__init__
calling my_function
in MyParameterisedClassDecorator.__call__
in MyLocalClassDecorator.__call__
do your decorating of args and kwargs here!
in my_function: non-parameterised class
my_function is an MyParameterisedClassDecorator so can access as such
type(my_function) = <class '__main__.MyParameterisedClassDecorator'>
my_function.func.parameter = False
my_function.func.state = 0
my_function.get_state() = (False, 0)


Decorating my_function with parameters...

In [6]:
@MyParameterisedClassDecorator(a_parameter=True)
def my_function(my_function_parameter):
    print("in my_function:", my_function_parameter)

my_function('parameterised class')
my_function('parameterised class')
print("my_function is an MyParameterisedClassDecorator.MyLocalClassDecorator object so can access as such")
print("type(my_function) =", type(my_function))
print("my_function.parameter = ", my_function.parameter)
print("my_function.state = ", my_function.some_state)
print("my_function.get_state() =", my_function.get_state())


in MyParameterisedClassDecorator.__init__
in MyParameterisedClassDecorator.__call__
in MyLocalClassDecorator.__init__
in MyLocalClassDecorator.__call__
do your decorating of args and kwargs here!
in my_function: parameterised class
in MyLocalClassDecorator.__call__
do your decorating of args and kwargs here!
in my_function: parameterised class
my_function is an MyParameterisedClassDecorator.MyLocalClassDecorator object so can access as such
type(my_function) = <class '__main__.MyParameterisedClassDecorator.MyLocalClassDecorator'>
my_function.parameter =  True
my_function.state =  2
my_function.get_state() = (True, 2)
